[TODO]: About Mnist

In [9]:
from core.utils import Utils

device = Utils.select_cuda_device()
device

Using mps device


'mps'

## Creating the data lodaers

In [1]:
import torchvision
from core.dataLoaders.fashion_mnist import Fashion_MNIST
import torchvision.transforms as transforms
# Load the Fashion MNIST dataset
transform = transforms.Compose([
    transforms.Resize((28, 28)),  # LeNet expects 32x32 input size
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

#instantiate the data class
data_cls = Fashion_MNIST(transforms=transform)

# create data loader
[train_loader, test_loader] = data_cls.create_loader()

train_data_sample_shape  = data_cls.view_sample_stats()
print(train_data_sample_shape)

/Users/srrthk/anaconda3/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/srrthk/anaconda3/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <CFED5F8E-EC3F-36FD-AAA3-2C6C7F8D3DD9> /Users/srrthk/anaconda3/lib/python3.11/site-packages/torchvision/image.so
  Expected in:     <13D54172-DB68-3BCA-9DFD-9BABA030E1E4> /Users/srrthk/anaconda3/lib/python3.11/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(



------------- Train Stats --------------

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64

------------- test Stats  --------------

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64

-------------    END      ---------------

torch.Size([64, 1, 28, 28])


## Initilaizing Model

In [2]:
from core.models.lenet import LeNet

model = LeNet(train_data_sample_shape[1], len(data_cls.labels))


In [13]:
model = model.to(device)
print(model)

LeNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (avg_pool1): AvgPool2d(kernel_size=(2, 2), stride=2, padding=0)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (avg_pool2): AvgPool2d(kernel_size=(2, 2), stride=2, padding=0)
  (conv3): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=120, out_features=84, bias=True)
  (output): Linear(in_features=84, out_features=10, bias=True)
)


## Initilalizing the loss and optimizer

In [14]:
import torch
import torch.nn as nn
from torch import optim

loss_model = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [15]:
# Training the model
num_epochs = 10



for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, (images, labels) in enumerate(train_loader): # Runs total data len/ batch size
        # Forward pass
        outputs = model.forward(images.to(device))
        loss = loss_model(outputs, labels.to(device))
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f'Epoch loss = {running_loss/len(train_loader)}')
     

Epoch loss = 0.35487533687179024
Epoch loss = 0.3455698481961481
Epoch loss = 0.33746371865431385
Epoch loss = 0.32997257550006737
Epoch loss = 0.3229989483475939
Epoch loss = 0.3164615281927052
Epoch loss = 0.3103039337119568
Epoch loss = 0.30448077917734445
Epoch loss = 0.2989546373200569
Epoch loss = 0.29369420957749587


## Testing on Test Data

In [18]:
# Evaluating the model
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images.to(device))
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels.to(device)).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy on the test set: {accuracy:.2f}%')


Accuracy on the test set: 87.33%
